In [1]:
 
import cv2
import pandas as pd
import numpy as np
import face_recognition
from datetime import datetime, date
import os
from tkinter import *
import csv
import turtle
from tkinter import messagebox

In [2]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)
        if len(encode) > 0:  # Check if any face is detected
            encodeList.append(encode[0])
        else:
            print("No face found in image")
    return encodeList

In [3]:
def display_csv():
    screen = turtle.Screen()
    screen.title("SECURITY")
    answer= screen.textinput(title=f"Security", prompt="Enter security key")
    if int(answer)==7689:
        screen.bye()
        file_path = "Book1.csv"  # Replace with your CSV file path
        try:
            df = pd.read_csv(file_path)
    
            # Create a new window to display the data
            data_window = Toplevel(window)
            data_window.title("CSV Data")
    
            # Create a text widget to display the data
            text_widget = Text(data_window)
            text_widget.pack(expand=True, fill="both")
    
            # Insert the data into the text widget
            text_widget.insert(END, df.to_string(index=False))
        except FileNotFoundError:
            messagebox.showerror("Error", "File not found")
    else:
        screen.bye()#to close window 
        messagebox.showinfo(title="CHECK",message="WRONG SECURITY CODE")

In [4]:
def markAttendance(name):
    with open('Book1.csv', 'r+') as f:  # r+ to read and write simultaneously
        myDataList = f.readlines()  # Read existing lines
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])

        # If the name is not already marked for the current session
        if name not in nameList:
            now = datetime.now()
            date1 = now.date()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString},{date1}')  # Append new entry

In [5]:
def face_recogition():
    # Path to the directory containing images
    path = "image"
    images = []
    classNames = []  # Initialize classNames here
    myList = os.listdir(path)  # Initialize myList before use
    print(myList)
    
    # Load images and class names
    for cl in myList:
        curImg = cv2.imread(f'{path}/{cl}')
        if curImg is None:
            print(f"Error loading image {cl}. Skipping...")
            continue  # Skip the image if it can't be loaded
        images.append(curImg)
        classNames.append(os.path.splitext(cl)[0])  # Remove file extension from name
    # Find encodings of known faces
    encodeListKnown = findEncodings(images)
    print(f'Encodings Complete, Number of Known Faces: {len(encodeListKnown)}')
    
    # Clear the CSV file at the start of the program
    with open('Book1.csv', 'w') as f:
        f.write('Name,Time,Date')  # Optionally write the headers if needed
    # Capture from default camera (0 for internal webcam, 1 for external)
    cap = cv2.VideoCapture(0)
    while True:
        success, img = cap.read()  # Read the frame from the webcam
        if not success:
            print("Failed to capture image")
            break
    
        imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)  # Resize frame for faster processing
        imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
        faceCurFrame = face_recognition.face_locations(imgS)  # Find face locations in the frame
        encodesCurFrame = face_recognition.face_encodings(imgS, faceCurFrame)  # Encode the detected faces
    
        for encodeFace, faceLoc in zip(encodesCurFrame, faceCurFrame):  # Loop through faces
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
            print(faceDis)
    
            matchIndex = np.argmin(faceDis)  # Get the index of the best match
            if matches[matchIndex]:
                name = classNames[matchIndex].upper()
                print(name)
    
                # Get face location and scale it back to the original size
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
    
                # Draw a rectangle around the face
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
    
                markAttendance(name)  # Mark the attendance
    
        cv2.imshow('Webcam', img)  # Display the frame
    
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
            break
    
    # Release the webcam and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

In [6]:
window=Tk()
PINK = "#e2979c"
RED = "#e7305b"
GREEN = "#9bdeac"
YELLOW = "#f7f5dd"
FONT_NAME = "Courier"
window.title("Attendence system")
window.config(padx=50,pady=50)

title_label=Label(text="Welcome to smart \n attendence system",font=(FONT_NAME,50,"bold"),fg=GREEN,bg=YELLOW)
title_label.grid(column=1,row=0)

canvas=Canvas(width=400,height=224,bg=YELLOW,highlightthickness=0)
img=PhotoImage(file="bg.png")
canvas.create_image(100,112,image=img)
canvas.grid(column=1,row=1)
start_button=Button(text="Take attendence",highlightthickness=0,command=face_recogition)
start_button.grid(column=0,row=2)

reset_button=Button(text="Show record",command=display_csv)
reset_button.grid(column=2,row=2)
exit_button=Button(text=" Exit ",command=window.destroy)
exit_button.grid(column=1,row=2)

window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\sumanyu singh\AppData\Local\Temp\ipykernel_28376\1657105089.py", line 5, in display_csv
    if int(answer)==7689:
       ^^^^^^^^^^^
TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'


['.ipynb_checkpoints', 'Azim Premji.jpg', 'Elon.jpg', 'k sivan.jpg', 'Ratan tata.jpeg', 'Ratan tata.jpg', 'sumanyu.jpg']
Error loading image .ipynb_checkpoints. Skipping...
Encodings Complete, Number of Known Faces: 6
[0.67477251 0.75669435 0.68004224 0.77421221 0.78197592 0.53074505]
SUMANYU
[0.63764896 0.79713222 0.66060294 0.76591557 0.77029916 0.47101182]
SUMANYU
[0.69514982 0.80698278 0.71932944 0.79611272 0.79787701 0.48411808]
SUMANYU
[0.69090688 0.81632707 0.71438916 0.78231726 0.78462135 0.52188369]
SUMANYU
[0.63851078 0.79844866 0.59570909 0.68627127 0.6882343  0.50977378]
SUMANYU
[0.68499813 0.78840204 0.64766649 0.72257672 0.73087627 0.4963316 ]
SUMANYU
